In [1]:
!pip install faiss-gpu > None
!pip install sentence_transformers > None
!pip install transformers > None

In [2]:
import pandas as pd
import re
from tqdm import tqdm
from collections import defaultdict
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer, util
from transformers import pipeline
import gc

In [3]:
df = pd.read_csv('/content/drive/MyDrive/resume_job_matching/Raw_Jobs.csv', delimiter=';')
df.head()

,№,id,title,salary,experience,job_type,description,key_skills,company,location,date_of_post,type
0,1,78577559,Главный механик,от 160000 до 160000 RUR,Более 6 лет,"Полная занятость,полный день","Вакансия компании: ООО ПК Предприятие ""ПИК"" ...",NaN,JCat.ru,Владивосток,27.03.2023,close
1,2,78693069,HTML-верстальщик,з/п не указана,От 1 года до 3 лет,"Полная занятость,полный день",В креативное агентство на полный рабочий день ...,NaN,Чугунова Ирина,NaN,29.03.2023,close
2,3,78945452,Тренер (направление профессионального обучения),з/п не указана,От 1 года до 3 лет,"Полная занятость,полный день",Наши предложения: самостоятельность в...,NaN,Перекрёсток,NaN,18.04.2023,close
3,4,79352008,Старший механик на КСПГ,з/п не указана,Более 6 лет,"Полная занятость,полный день",Обязанности: _Обеспечение технически ...,NaN,Газпром гелий сервис,NaN,17.04.2023,close
4,5,79406472,System Analyst Trainee,з/п не указана,Нет опыта,"Стажировка,полный день",IT-компания Aston - компания по разработке про...,"UML,SQL,Scrum,Retail,Базы данных,Kanban,Waterf...",Aston,NaN,17.04.2023,close


In [4]:
valid_comment_pattern = r"программист|разработчик|developer"

In [5]:
df['is_valid_pattern'] = df['title'].apply(lambda x: True if not isinstance(x, float) and re.match(valid_comment_pattern, x.lower()) is not None else False)


In [6]:
df = df[df.is_valid_pattern == True]


In [7]:
#парсим описание вакансии как можем
def extract_information(job_description):
    responsibilities_pattern = r'\b(?:вам предстоит|вы будете|задачи|обязанности|чем необходимо заниматься)\b:(.*?)(?=\b(?:требования|требования к кандидатам|мы предлагаем|условия|требования:|мы ожидаем|наши пожелания|будет плюсом|будет преимуществом)\b|$)'
    requirements_pattern = r'\b(?:для нас важно|ждем от вас|мы ожидаем|наши пожелания|необходимые навыки|требования|требования к кандидатам|будет плюсом|будет преимуществом)\b:(.*?)(?=\b(?:мы предлагаем|условия|место работы|длительность рабочего дня)\b|$)'
    conditions_pattern = r'\b(?:мы предлагаем|условия)\b:(.*?)(?=\b(?:в ответ на оставленный отклик|образование)\b|$)'

    responsibilities_match = re.search(responsibilities_pattern, job_description, re.IGNORECASE | re.DOTALL)
    requirements_match = re.search(requirements_pattern, job_description, re.IGNORECASE | re.DOTALL)
    conditions_match = re.search(conditions_pattern, job_description, re.IGNORECASE | re.DOTALL)

    responsibilities = responsibilities_match.group(1).strip() if responsibilities_match else None
    requirements = requirements_match.group(1).strip() if requirements_match else None
    conditions = conditions_match.group(1).strip() if conditions_match else None

    return responsibilities, requirements, conditions

In [8]:
responsabilities, req, cond = list(), list(), list()

In [9]:
for row in tqdm(df['description']):
    responsability, requirements, conditions = extract_information(row)
    responsabilities.append(responsability)
    req.append(requirements)
    cond.append(conditions)

100%|██████████| 1219/1219 [00:00<00:00, 3754.08it/s]


In [10]:
df['responsabilities'] = responsabilities
df['requirements'] = req
df['conditions'] = cond

In [11]:
del responsabilities
del req
del cond
gc.collect()

93

In [12]:
vacancy_text = defaultdict(str)
for idx, row in df.iterrows():
    vacancy_text[idx]  = f"Профессия: {row['title']}. Ключевые навыки: {row['key_skills']}. Обязанности: {row['responsabilities']}. Требования: {row['requirements']}. Условия {row['conditions']}"

In [13]:
df['text'] = df.index.map(vacancy_text)

# Начинаем поиск

In [14]:
model = SentenceTransformer('intfloat/multilingual-e5-large')

.gitattributes:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

In [23]:
query = """Программист C++, Web-программист, разработчик аппаратуры,.Опыт работы 5 лет 8 месяцев  Март 2019 — по настоящее время 3 месяца Удаленная работа Обнинск Web-программист Доработка сайта www.autolarek24.ru  Октябрь 2013 — по настоящее время 5 лет 8 месяцев АО «ГНЦ РФ – ФЭИ» Обнинск , www.ippe.ru Энергетика ... Атомная энергетика (генерация электроэнергии, АЭС) Инженер-программист Разработка ПО для операционных систем жесткого реального времени. Разработка драйверов, многопоточных приложений, клиент-серверных приложений. Разработка микропроцессорных систем. Работа с промышленными интерфейсами и протоколами: ModBus, CAN, RS422/485. Разработка документации ПО. Работа с измерительной аппаратурой с очень высокой точностью измерения и написание ПО для мониторинга и фильтрации измерений, а так же участие в разработке, настройке и тестировании данной аппаратуры.  Июнь 2017 — Ноябрь  2017 6 месяцев ООО "Персона" Обнинск Информационные технологии, системная интеграция, интернет ... Разработка программного обеспечения Веб-разработчик Разработка веб-сайтов, концепция MVC, командная разработка (Git), Laravel. Проект www.combeep.com"""

In [16]:
encoded_data = model.encode(df.text.tolist())
encoded_data = np.asarray(encoded_data.astype('float32'))
index = faiss.IndexIDMap(faiss.IndexFlatIP(1024))
index.add_with_ids(encoded_data, np.array(range(0, len(df))))
faiss.write_index(index, 'vacancy.index')

#1. Faiss - отбираем кандидатов

In [37]:
from time import time
#todo: поправить на суммаризацию, если текст длинный?
def fetch_vacancy_info(dataframe_idx):
    info = df.iloc[dataframe_idx]
    meta_dict = dict()
    meta_dict['title'] = info['title']
    meta_dict['text'] = info['text'][:1024]
    return meta_dict

def search(query, top_k, index, model):
    t=time()
    query_vector = model.encode([query])
    top_k = index.search(query_vector, top_k)
    print('>>>> Results in Total Time: {}'.format(time()-t))
    top_k_ids = top_k[1].tolist()[0]
    top_k_ids = list(np.unique(top_k_ids))
    results =  [fetch_vacancy_info(idx) for idx in top_k_ids]
    return results

In [38]:
from pprint import pprint
results=search(query, top_k=5, index=index, model=model)
print("\n")
for result in results:
    print('\t',result)

>>>> Results in Total Time: 0.13213086128234863


	 {'title': 'Программист-разработчик', 'text': 'Профессия: Программист-разработчик. Ключевые навыки: Python,Работа с базами данных,C++,Алгоритмы и структуры данных. Обязанности: разрабатывать оборудование и по для двухстороннего стрима данных в последовательных и распределенных сетях, тренировка алгоритмов под текущие нужды.   Рассматриваются только кандидаты для оффлайн работы   Договор, пятидневка с 9 до18 центральных район. Требования: опыт работы с микроконтроллерами, опыт самостоятельной разработки, электроника   Обязанности: разрабатывать оборудование и по для двухстороннего стрима данных в последовательных и распределенных сетях, тренировка алгоритмов под текущие нужды.   Рассматриваются только кандидаты для оффлайн работы   Договор, пятидневка с 9 до18 центральных район. Условия None'}
	 {'title': 'Программист C/С++', 'text': 'Профессия: Программист C/С++. Ключевые навыки: Linux,C/C++,C++,CMake,Git,Boost,doctest,liburing,nfs,net

## 2. Cross Encoder

In [39]:
from sentence_transformers import CrossEncoder
cross_model = CrossEncoder('PitKoro/cross-encoder-ru-msmarco-passage', max_length=2048)

In [40]:
def cross_score(model_inputs):
    scores = cross_model.predict(model_inputs)
    return scores

model_inputs = [[query,item['text']] for item in results]
scores = cross_score(model_inputs)
#Sort the scores in decreasing order
ranked_results = [{'Title': inp['title'], 'Text':inp['text'], 'Score': score} for inp, score in zip(results, scores)]
ranked_results = sorted(ranked_results, key=lambda x: x['Score'], reverse=True)

In [41]:
print("\n")
for result in ranked_results:
    print('\t',pprint(result))



{'Score': 0.49219885,
 'Text': 'Профессия: Программист С++. Ключевые навыки: Qt,Debian,Astra '
         'Linux,SQL,STL,Multithread Programming,MS Visual C++,OPC,TCP/IP. '
         'Обязанности: .      Разработка специализированного '
         'кросс-платформенного клиент-серверного ПО в среде QT. Требования: '
         'Обязательно:        опыт разработки ПО на базе клиент-серверной '
         'архитектуры под Linux     опыт разработки с использованием QT     '
         'практические навыки написания многопоточного кода     '
         'Желательно:        опыт разработки под Windows (MS Visual C++)     '
         'знание протоколов OPC UA/DA, MODBUS, SNMP, MEC 870-5-101/104      '
         'знание основных сетевых протоколов, технологий, шифрования (TCP/IP, '
         'UDP, ARP, ICMP, HTTP, HTTPS)      опыт написания библиотек     '
         'знание winapi, SQL, PostgreSQL, MS SQL Server, ODBC      английский '
         'язык на уровне чтения документации.. Условия Трудоустройство по 